## LASSO Regression (L1)
Es un tipo popular de regresión lineal regularizada que incluye una penalización L1. Esto tiene el efecto de reducir los coeficientes para aquellas variables de entrada que no contribuyen mucho a la tarea de predicción. Esta penalización permite que algunos valores de coeficiente lleguen al valor de cero, lo que permite que las variables de entrada se eliminen efectivamente del modelo, proporcionando un tipo de selección automática de características.<br>
Una penalización L1 minimiza el tamaño de todos los coeficientes y permite que algunos coeficientes se minimicen al valor cero, lo que elimina el predictor del modelo. Una penalización L1 minimiza el tamaño de todos los coeficientes y permite que cualquier coeficiente tome el valor cero, eliminando de manera efectiva las características de entrada del modelo

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.random import check_random_state
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

### Ejemplo LASSO Regression
Se aplicará LASSO Regression para predecir la distancia entre un par de puntos (x,y) y (x1,y1), uno de ellos constante.$$ d = \sqrt{(x1-x)^2 + (y1 - y)^2} $$

### Generar el conjunto de datos
Para generar el conjunto de datos se calculará la distancia entre un conjunto de 5000 puntos (x,y) aleatorios, así, el modelo de regresión se ajustará a una proporción del 80% del conjunto y el 20% restante estará destinado a la predicción.

In [2]:
def calcular_distancia(X,Y,XF,YF):
    return np.round(np.sqrt(np.power(XF-X,2)+np.power(YF-Y,2)),2)

XF = 5
YF = 12.5

rng = check_random_state(0)

X = np.round(rng.uniform(1, 11, 10000).reshape(5000, 2),2)
XFs=[[XF] for i in range(5000)]
YFs=[[YF] for i in range(5000)]
X=np.append(X, XFs, axis=1)
X=np.append(X, YFs, axis=1)
Y = calcular_distancia(X[:, 0],X[:, 1],XF,YF)
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

### Crear y ajustar el modelo
Número de divisiones (K-Fold) del conjunto de datos para el entrenamiento (n_splits=10) inicialmente tomará el prier subconjunto como validación y los demás de entrenamiento, luego el segundo bloque será de validación y el resto de prueba y así sucesivamente; en este caso el proceso se repetirá 3 veces (n_repeats=3).

In [4]:
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, x_train, y_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
scores = np.abs(scores)
print("Scores: ")
print(scores)
print()
print("Media de RMSE para los scores = ",np.mean(scores))

Scores: 
[0.66052933 0.63477281 0.59270957 0.59997847 0.61768703 0.65049786
 0.68374985 0.65502851 0.63115152 0.56495541 0.6879637  0.5805683
 0.61976596 0.62491272 0.56736454 0.68938526 0.61045108 0.60914042
 0.63273663 0.66910765 0.64452186 0.70752317 0.60087634 0.62925951
 0.653732   0.63897815 0.68607939 0.56831762 0.57621467 0.59644754]

Media de RMSE para los scores =  0.62948022916841


### Métricas
Se usará el GridSearch para buscar el valor optimizado de alpha, y luego, la métrica utilizada para medir el error sera RMSE.

In [7]:
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)

search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
results = search.fit(x_train, y_train)

print('RMSE para el entrenamiento: %.3f' % np.sqrt(np.abs(results.best_score_)))
print('Config: %s' % results.best_params_)

tst_predict = search.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test,tst_predict))
print("RMSE para las predicciones = ",rmse)

RMSE para el entrenamiento: 0.625
Config: {'alpha': 0.0}
RMSE para las predicciones =  0.6015157314732367


C:\Users\JVelez\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\JVelez\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\JVelez\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 779.2141895586944, tolerance: 2.9079256097975015
  positive)
